In this notebook we will focus on Extraction of information from pdf using GPT Model

In [1]:
%autosave 300
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir(
    "/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/unstructured_data_experiments"
)
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/unstructured_data_experiments


In [3]:
from pdf2image import convert_from_path
import os

In [4]:
def convert_pdf_to_images(pdf_path, dest_folder):
    """
    Converts each page of a PDF into JPEG images and saves them in a directory named after the PDF file.

    Args:
    - pdf_path (str): Path to the PDF file.

    Returns:
    - list: List of image file paths saved.
    """
    # Create a directory based on the PDF filename
    pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
    output_dir = os.path.join(f"{dest_folder}/{pdf_name}_images")
    os.makedirs(output_dir, exist_ok=True)

    # Convert each page of the PDF into images
    images = convert_from_path(pdf_path)
    saved_image_paths = []
    for i, img in enumerate(images):
        image_path = os.path.join(output_dir, f"page{i}.jpeg")
        img.save(image_path, "JPEG")
        saved_image_paths.append(image_path)

    return saved_image_paths

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from dotenv import load_dotenv, find_dotenv
import base64, requests, json
from datetime import datetime
from langchain.schema import Document
import re

In [6]:
load_dotenv(find_dotenv("dev.env"))

True

In [7]:
print(os.getenv("AZURE_OPENAI_API_KEY"))
print(os.getenv("AZURE_OPENAI_ENDPOINT"))
print(os.getenv("OPENAI_API_VERSION"))

64a128e1a82f40888c77f200c6e5e661
https://brewdatgbgaighqtechopenai01ncud.openai.azure.com/
2024-02-15-preview


In [8]:
llm_model = llm = AzureChatOpenAI(
    azure_deployment=os.getenv("CHAT_ENGINE_GPT4_DEPLOYMENT_NAME"),
    model_name=os.getenv("CHAT_ENGINE_GPT4_MODEL_NAME"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_type=os.getenv("OPENAI_API_TYPE"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    temperature=0.0,
    request_timeout=45,
    max_retries=5,
    seed=1234,
    top_p=0.0001,
)

In [10]:
await llm_model.ainvoke("Hello, how are you?")

AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 13, 'total_tokens': 43}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-2acb6d79-2e51-4638-a8a2-0629e15004d0-0', usage_metadata={'input_tokens': 13, 'output_tokens': 30, 'total_to

In [11]:
# # Read and encode the image file
# image_path = r"image_extracts/1. MSA Gateware (Enteron) 1_images/page23.jpg"
# encoded_image = base64.b64encode(open(image_path, 'rb').read()).decode('utf-8')
# print(len(encoded_image))

In [12]:
# import tiktoken

# def num_tokens_from_string(string: str, model_name: str = 'gpt-4o') -> int:
#     encoding = tiktoken.encoding_for_model(model_name)
#     num_tokens = len(encoding.encode(string))
#     return num_tokens

In [13]:
# num_tokens_from_string(encoded_image)

In [14]:
# SYSTEM_PROMPT = """
# From the above image extract the text as is and export the information from chart into tabular format so that one can understand meaning of chart and can reproduce this chart from tabular data and insert the tabular information in the same place where chart is present in the image. Provide special focus while extracting the data from the tables so that the table structure is maintained and while you are extracting the data from the tables, provide the data in structure 2D format.If there are hierarchies columns in the table, provide the data in the form of right table structure.
# """

# HUMAN_PROMPT="""
# The raw image url in encoded format is "data:image/jpeg;base64,{encoded_image}
# """

In [15]:
# input_variables = ["encoded_image"]

# # Create messages for prompt
# messages = [
#     SystemMessagePromptTemplate.from_template(SYSTEM_PROMPT),
#     HumanMessagePromptTemplate.from_template(HUMAN_PROMPT),
# ]

# # Create the final prompt template
# prompt = ChatPromptTemplate(input_variables=input_variables, messages=messages)

# chain = prompt | llm | StrOutputParser()

In [16]:
# out = chain.invoke(encoded_image)
# print(out)

In [17]:
# # Configuration
# headers = {
#     "Content-Type": "application/json",
#     "api-key": os.getenv("AZURE_OPENAI_API_KEY"),
# }

# # Payload for the request
# payload = {
#     "messages": [
#         {
#             "role": "system",
#             "content": [
#                 {
#                     "type": "text",
#                     "text": """You are an AI assistant specializing in extracting information from images containing three main components: text, tables, and figures. Your tasks are as follows:
#                 * Extract the text exactly as it appears in the image.
#                 * Extract data from tables while preserving the table structure, providing the data in a structured 2D format, and maintaining hierarchical columns if present.
#                 * Extract and describe the information from figures in plain text format, making it understandable and reproducible from the extracted data.

#                 The final output should be in plain text format, containing only the extracted information. You can use the image URL provided to extract the information.""",
#                 }
#             ],
#         },
#         {
#             "role": "user",
#             "content": [
#                 {
#                     "type": "image_url",
#                     "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"},
#                 }
#             ],
#         },
#     ],
#     "temperature": 0.0,
#     "top_p": 1e-5,
#     "seed": 1234,
# }

# # GPT-4 Vision endpoint
# GPT4V_ENDPOINT = f"{os.getenv('AZURE_OPENAI_ENDPOINT')}openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"

In [18]:
# # Send request
# try:
#     response = requests.post(GPT4V_ENDPOINT, headers=headers, json=payload)
#     response.raise_for_status()
# except Exception as e:
#     raise SystemExit(f"Failed to make the request. Error: {e}")

In [19]:
# response.json()

In [20]:
# print(response.json()['choices'][0]['message']['content'])

In [21]:
# print(response.json()['usage']['completion_tokens'])

In [22]:
# # get current time
# now = datetime.now()
# print(now)

In [23]:
# op = response.json()['choices'][0]['message']['content']
# print(op)
# # calculate the token count of output
# num_tokens_from_string(op)

#### __Modularize entire code__
* We will modularize the entire code into functions
* We will try extarcting information from pdf and save it in langchain document class
* We will parallelize the entire code using joblib

In [11]:
# Example usage:
pdf_file = os.listdir("data")[0]
pdf_path = os.path.join(os.getcwd(), "data", pdf_file)
saved_paths = convert_pdf_to_images(pdf_path, "image_extracts")
print("Images saved to:", saved_paths)

Images saved to: ['image_extracts/1. MSA Gateware (Enteron) 1_images/page0.jpg', 'image_extracts/1. MSA Gateware (Enteron) 1_images/page1.jpg', 'image_extracts/1. MSA Gateware (Enteron) 1_images/page2.jpg', 'image_extracts/1. MSA Gateware (Enteron) 1_images/page3.jpg', 'image_extracts/1. MSA Gateware (Enteron) 1_images/page4.jpg', 'image_extracts/1. MSA Gateware (Enteron) 1_images/page5.jpg', 'image_extracts/1. MSA Gateware (Enteron) 1_images/page6.jpg', 'image_extracts/1. MSA Gateware (Enteron) 1_images/page7.jpg', 'image_extracts/1. MSA Gateware (Enteron) 1_images/page8.jpg', 'image_extracts/1. MSA Gateware (Enteron) 1_images/page9.jpg', 'image_extracts/1. MSA Gateware (Enteron) 1_images/page10.jpg', 'image_extracts/1. MSA Gateware (Enteron) 1_images/page11.jpg', 'image_extracts/1. MSA Gateware (Enteron) 1_images/page12.jpg', 'image_extracts/1. MSA Gateware (Enteron) 1_images/page13.jpg', 'image_extracts/1. MSA Gateware (Enteron) 1_images/page14.jpg', 'image_extracts/1. MSA Gateware 

In [12]:
# we have to parallelize the image extraction process
file_name = pdf_file.replace(".pdf", "")
docs_ls = os.listdir(f"image_extracts/{file_name}_images")
docs_ls = [os.path.join(f"image_extracts/{file_name}_images", doc) for doc in docs_ls]
print(len(docs_ls))

64


In [13]:
def encode_image(image_path):
    encoded_image = base64.b64encode(open(image_path, "rb").read()).decode("utf-8")
    return encoded_image

In [14]:
def get_gpt_model_infra(
    api_key,
    endpoint,
):
    # headers for the request
    headers = {
        "Content-Type": "application/json",
        "api-key": api_key,
    }

    # GPT-4 Vision endpoint
    GPT4V_ENDPOINT = f"{endpoint}openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"

    return headers, GPT4V_ENDPOINT

In [15]:
def get_gpt_model_payload(
    encoded_image,
    temperature=0.0,
    top_p=1e-5,
    seed=1234,
):
    # Payload for the request
    payload = {
        "messages": [
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": """You are an AI assistant specializing in extracting information from images containing three main components: text, tables, and figures. Your tasks are as follows:
                    * Extract the text exactly as it appears in the image.
                    * Extract data from tables while preserving the table structure, providing the data in a structured 2D format, and maintaining hierarchical columns if present.
                    * Extract and describe the information from figures in plain text format, making it understandable and reproducible from the extracted data.

                    The final output should be in plain text format, containing only the extracted information. You can use the image URL provided to extract the information.""",
                    }
                ],
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"},
                    }
                ],
            },
        ],
        "temperature": temperature,
        "top_p": top_p,
        "seed": seed,
    }

    return payload

In [16]:
def invoke_gpt_model_response(headers, endpoint, payload):
    try:
        response = requests.post(endpoint, headers=headers, json=payload)
        response.raise_for_status()
    except Exception as e:
        raise SystemExit(f"Failed to make the request. Error: {e}")

    return response.json()

In [17]:
def extract_text_from_image(image_path, api_key, endpoint):
    encoded_image = encode_image(image_path)
    headers, GPT4V_ENDPOINT = get_gpt_model_infra(api_key, endpoint)
    payload = get_gpt_model_payload(encoded_image)
    response = invoke_gpt_model_response(headers, GPT4V_ENDPOINT, payload)
    return response

In [18]:
def lc_document_from_response(response, image_path):
    pg_content = response["choices"][0]["message"]["content"]
    token_size = response["usage"]["completion_tokens"]
    curr_time = datetime.now()
    pdf_name = re.findall(r"(.*)_images\b", image_path.split("/")[-2])[0]
    page_num = re.findall(r"page(\d+).jpg", image_path.split("/")[-1])[0]

    # Create a new document with page number and additional metadata
    doc = Document(
        page_content=pg_content,
        metadata={
            "source": pdf_name,
            "page_number": int(page_num),
            "token_size": token_size,
            "timestamp": str(curr_time),
        },
    )

    return doc

In [19]:
def final_image_extraction_pipeline(image_path, api_key, endpoint):
    response = extract_text_from_image(image_path, api_key, endpoint)
    doc = lc_document_from_response(response, image_path)
    return doc

In [22]:
doc_cls = final_image_extraction_pipeline(
    docs_ls[0], os.getenv("AZURE_OPENAI_API_KEY"), os.getenv("AZURE_OPENAI_ENDPOINT")
)

KeyboardInterrupt: 

In [ ]:
doc_cls

NameError: name 'doc' is not defined